In [1]:
!pip install sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
import sqlite3
import pandas as pd
import math

class CricketDataPipeline:
    def __init__(self, input_file, batter_output_file, bowler_output_file, database_file):
        self.input_file = input_file
        self.batter_output_file = batter_output_file
        self.bowler_output_file = bowler_output_file
        self.database_file = database_file

    def read_input_data(self):
        return pd.read_csv(self.input_file, sep=',')

    def create_database_connection(self):
        return sqlite3.connect(self.database_file)

    def save_to_csv(self, data, output_file):
        data.to_csv(output_file, sep='\t', index=False, header=True)

    def extract_batter_statistics(self, conn):
        sql_string = """
            SELECT event_id as match_id, batter_id as batsman_id, batsman_striker_full_name as full_name,
                   batsman_striker_team_id as team_id, bowler_team_id as opposition, batsman_striker_name,
                   batsman_striker_team_name as team_name, MAX(batter_runs) as runs, batter_balls_faced as balls_faced
            FROM cricket_table
            GROUP BY batter_id
            ORDER BY event_id DESC
        """
        return pd.read_sql(sql_string, conn)

    def extract_bowler_statistics(self, conn):
        sql_string = """
            SELECT MAX(id), event_id as match, bowler_team_id as team_id, bowler_team_name as team_name,
                   CAST(innings_remaining_runs as int) as outcome
            FROM cricket_table
            GROUP BY event_id
        """
        return pd.read_sql(sql_string, conn)

    def run_pipeline(self):
        data = self.read_input_data()

        with self.create_database_connection() as conn:
            try:
              data.to_sql('cricket_table', conn)
            except:
              pass
            batter_data = self.extract_batter_statistics(conn)
            bowler_data = self.extract_bowler_statistics(conn)

        self.save_to_csv(batter_data, self.batter_output_file)
        self.save_to_csv(bowler_data, self.bowler_output_file)


class RatingCalculator:
    def __init__(self):
        self.players_ranking = {}

    def probability(self, rating1, rating2):
        return 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

    def calculate_rating(self, player_rating, win, batter_runs, batter_balls_faced, team_rating, opposition_rating):
        k = 0.02
        p = 0.01
        prob = self.probability(team_rating, opposition_rating)

        if win == 1:
            if player_rating + k * (1 - prob) + p * batter_runs / batter_balls_faced <= 1000:
                player_rating = player_rating + k * (1 - prob) + p * batter_runs / batter_balls_faced
            else:
                player_rating = 1000
        else:
            if player_rating - k * (1 - prob) - p * batter_runs / batter_balls_faced >= 0:
                player_rating = player_rating - k * (1 - prob) - p * batter_runs / batter_balls_faced
            else:
                player_rating = 0

        return player_rating


class DataAnalyzer:
    def __init__(self, database_file):
        self.conn = sqlite3.connect(database_file)

    def load_data(self, table_name):
        query = f"SELECT * FROM {table_name}"
        return pd.read_sql(query, self.conn)

    def store_data_to_sqlite(self, table_name, data):
        try:
          data.to_sql(table_name, self.conn)
        except:
          pass

    def generate_word_cloud(self, data, column_name):
        text = " ".join(value for value in data[column_name])
        word_cloud = WordCloud(collocations=False, background_color='white').generate(text)
        plt.figure(figsize=(10, 6))
        plt.title('Players with Maximum Runs', fontweight='bold', color='blue')
        plt.imshow(word_cloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()


class MainApplication:
    def __init__(self, input_file, batter_output_file, bowler_output_file, database_file):
        self.pipeline = CricketDataPipeline(input_file, batter_output_file, bowler_output_file, database_file)
        self.rating_calculator = RatingCalculator()
        self.data_analyzer = DataAnalyzer(database_file)

    def run(self):
        self.pipeline.run_pipeline()

        batter_data = self.data_analyzer.load_data('batter_table')

        total_matches = len(batter_data['match_id'].unique())
        print('Total Matches:', total_matches)

        distinct_players = len(batter_data['batsman_id'].unique())
        print('Distinct Player IDs:', distinct_players)

        player_stats = batter_data.groupby('batsman_id').agg(
            {'runs': 'sum', 'balls_faced': 'sum', 'full_name': 'first', 'result': ['count', 'sum']}
        )
        player_stats.columns = ['runs', 'balls', 'matches_played', 'matches_won']
        player_stats = player_stats.dropna()

        player_stats.head(5).style.background_gradient(cmap='Dark2_r')
        player_stats.describe().style.background_gradient(cmap='Dark2')

        player_stats['runs'].hist(density=1, color='black', alpha=0.7)
        player_stats['balls'].hist(density=1, color='black', alpha=0.7)

        self.data_analyzer.generate_word_cloud(player_stats, 'full_name')

        r = [50] * len(batter_data)
        team_rankings = dict(zip(batter_data['team_id'], r))
        players_ranking = dict(zip(batter_data['batsman_id'], r))

        for i in batter_data.index:
            try:
                batsman_id = batter_data.loc[i, 'batsman_id']
                batter_team_id = batter_data.loc[i, 'team_id']
                opposition = batter_data.loc[i, 'opposition']
                batter_runs = batter_data.loc[i, 'runs']
                batter_balls_faced = batter_data.loc[i, 'balls_faced']
                batter_data_result = batter_data.loc[i, 'result']
                team_ranking = team_rankings[batter_team_id]
                opposition_team_ranking = team_rankings[opposition]
                player_ranking = players_ranking[batsman_id]

                win = batter_data.loc[i, 'result']
                new_ranking = self.rating_calculator.calculate_rating(player_ranking, win, batter_runs,
                                                                      batter_balls_faced, team_ranking,
                                                                      opposition_team_ranking)
                players_ranking[batsman_id] = new_ranking
            except:
                pass

        print(players_ranking)


# Define the file paths
input_file = '/content/drive/MyDrive/global/formatted_bbb_df.csv'
batter_output_file = '/content/drive/MyDrive/global/batter_final_data.csv'
bowler_output_file = '/content/drive/MyDrive/global/bowler_final_data.csv'
database_file = 'hello.db'

app = MainApplication(input_file, batter_output_file, bowler_output_file, database_file)
app.run()

DatabaseError: ignored